<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=6edbab9c41805b5129b93976541f47ee35f0959e1db99ab3ebf4c577f3fbcd63
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# conviction distribution
mypf['Conviction'].value_counts()

M    79
L    65
H    37
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0.0    93
1.0    83
Name: StarStock, dtype: int64

In [7]:
# v40/v40n stocks
mypf['Criteria'].value_counts()

V40N    39
V40     38
2T3Y    10
V200     6
SS       6
TBD      2
Name: Criteria, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

1    165
0     16
Name: LatestQtr, dtype: int64

In [9]:
# star + v40/v40n stocks
mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]['StarStock'].value_counts()

1.0    41
0.0    36
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    144
1     37
Name: InPortfolio, dtype: int64

In [11]:
# star stocks in portfolio
mypf[(mypf['InPortfolio'] == 1)]['StarStock'].value_counts()

1.0    22
0.0    13
Name: StarStock, dtype: int64

In [12]:
# criteria of portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Criteria'].value_counts()

V40     11
2T3Y    10
V40N     8
SS       5
TBD      2
V200     1
Name: Criteria, dtype: int64

In [13]:
# strategies in portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Strategy'].value_counts()

ATH     23
2T3Y    14
Name: Strategy, dtype: int64

In [14]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [15]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000

  return current_pe


In [16]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [17]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-16 00:00:00+05:30,18286.0,^NSEI,2.92,0.57,0.44,18812.0,7610.0,4.70,3.0
2023-05-23 00:00:00+05:30,18348.0,^NSEI,3.04,0.49,0.42,18812.0,7610.0,4.14,3.0
2023-05-30 00:00:00+05:30,18634.0,^NSEI,4.48,0.38,0.61,18812.0,7610.0,1.59,1.0
2023-06-06 00:00:00+05:30,18599.0,^NSEI,4.14,0.30,0.85,18812.0,7610.0,1.90,1.0
2023-06-13 00:00:00+05:30,18716.0,^NSEI,4.67,0.26,0.90,18812.0,7610.0,0.86,1.0


In [18]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [19]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [20]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 37


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,994.0,-37.50,-4.40,-2.61,97.08,334.0,1225.87,45.2,19.9,M,56.0,1,0.0,2T3Y,2T3Y
VALIANTORG.NS,562.0,-1.64,0.00,2.53,87.63,229.0,542.52,24.0,18.0,M,25.0,0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,313.0,-2.43,0.00,0.65,76.02,209.0,317.48,35.4,49.2,M,-39.0,1,0.0,2T3Y,2T3Y
LUXIND.NS,1483.0,-4.47,-1.03,1.94,82.85,207.0,1493.72,21.4,32.4,M,-51.0,1,0.0,V40N,2T3Y
BANDHANBNK.NS,262.0,7.67,-0.41,2.54,81.02,176.0,253.98,NaN,1000.0,L,NaN,1,1.0,2T3Y,2T3Y
NIACL.NS,119.0,13.00,0.95,1.79,82.85,166.0,117.07,NaN,18.7,M,NaN,1,0.0,TBD,ATH
AAVAS.NS,1350.0,-26.95,-2.27,-2.08,72.08,137.0,1390.60,NaN,24.8,M,NaN,1,1.0,2T3Y,2T3Y
TEAMLEASE.NS,2350.0,-8.53,-1.99,0.09,77.11,125.0,2279.93,NaN,35.4,M,NaN,1,1.0,V40N,2T3Y
INDIGOPNTS.NS,1425.0,8.95,-0.31,3.24,79.28,118.0,1407.73,104.9,51.4,M,51.0,1,1.0,V40N,2T3Y


In [21]:
# top stocks from portfolio for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,994.0,-37.50,-4.40,-2.61,97.08,334.0,1225.87,45.2,19.9,M,56.0,1,0.0,2T3Y,2T3Y
AAVAS.NS,1350.0,-26.95,-2.27,-2.08,72.08,137.0,1390.60,NaN,24.8,M,NaN,1,1.0,2T3Y,2T3Y
CLEAN.NS,1340.0,-11.70,-0.86,0.42,92.83,98.0,1396.49,NaN,48.2,L,NaN,0,NaN,2T3Y,2T3Y
PAGEIND.NS,38387.0,-11.49,-1.17,0.24,40.48,39.0,39325.03,97.7,74.9,H,23.0,1,1.0,V40,ATH
FINEORG.NS,4879.0,-10.43,-1.25,1.47,35.86,47.0,4697.43,NaN,24.2,M,NaN,1,1.0,V40N,ATH
TEAMLEASE.NS,2350.0,-8.53,-1.99,0.09,77.11,125.0,2279.93,NaN,35.4,M,NaN,1,1.0,V40N,2T3Y
INFY.NS,1305.0,-7.85,-0.92,-0.47,39.69,42.0,1263.00,29.0,22.6,H,22.0,1,1.0,V40,ATH
BAYERCROP.NS,4237.0,-5.87,-1.02,0.75,54.54,42.0,4185.24,NaN,387.0,M,NaN,1,1.0,V40N,ATH
JUBLFOOD.NS,485.0,-5.83,-0.97,0.87,59.97,86.0,471.56,90.5,90.6,H,-0.0,1,1.0,SS,ATH


In [22]:
# top 10 from portfolio for SIP
tmp_df.sort_values(by = 'Upside%ATH', ascending=False).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,994.0,-37.50,-4.40,-2.61,97.08,334.0,1225.87,45.2,19.9,M,56.0,1,0.0,2T3Y,2T3Y
VALIANTORG.NS,562.0,-1.64,0.00,2.53,87.63,229.0,542.52,24.0,18.0,M,25.0,0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,313.0,-2.43,0.00,0.65,76.02,209.0,317.48,35.4,49.2,M,-39.0,1,0.0,2T3Y,2T3Y
LUXIND.NS,1483.0,-4.47,-1.03,1.94,82.85,207.0,1493.72,21.4,32.4,M,-51.0,1,0.0,V40N,2T3Y
BANDHANBNK.NS,262.0,7.67,-0.41,2.54,81.02,176.0,253.98,NaN,1000.0,L,NaN,1,1.0,2T3Y,2T3Y
NIACL.NS,119.0,13.00,0.95,1.79,82.85,166.0,117.07,NaN,18.7,M,NaN,1,0.0,TBD,ATH
AAVAS.NS,1350.0,-26.95,-2.27,-2.08,72.08,137.0,1390.60,NaN,24.8,M,NaN,1,1.0,2T3Y,2T3Y


In [23]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PIDILITIND.NS,2687.0,5.57,-0.08,1.16,10.73,8.0,2504.06,94.6,107.0,H,-13.0,1,1.0,V40,ATH
ASIANPAINT.NS,3263.0,7.58,-0.40,1.40,11.42,9.0,3091.34,87.9,75.7,H,14.0,1,1.0,V40,ATH
BAJFINANCE.NS,7167.0,8.80,-0.20,2.15,12.07,10.0,6602.15,52.6,37.7,H,28.0,1,1.0,V40,ATH
RELIANCE.NS,2521.0,2.27,-0.28,1.03,14.92,12.0,2467.15,29.1,25.6,H,12.0,1,1.0,V40,ATH
AMBUJACEM.NS,459.0,-0.23,0.65,2.21,27.86,28.0,424.02,23.9,32.6,L,-36.0,0,0.0,TBD,ATH
BAJAJFINSV.NS,1486.0,-1.25,-0.47,1.51,28.10,28.0,1454.93,39.2,36.9,M,6.0,1,1.0,V40,ATH
ICICIGI.NS,1220.0,5.89,-0.17,1.35,41.85,32.0,1137.80,46.2,1000.0,M,-2065.0,1,1.0,V40,ATH


In [24]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [25]:
# star + v40/v40n stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 26)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
SUNTV.NS,448.0,-4.51,0.00,0.69,777.0,249.0,62.31,73.0,NaN,NaN,M,1,1.0,V40N,NaN
RAJESHEXPO.NS,577.0,-13.83,-0.15,-0.69,949.0,440.0,73.08,64.0,NaN,NaN,M,1,1.0,V40N,NaN
DIXON.NS,4316.0,18.24,0.14,3.88,5750.0,374.0,26.67,33.0,NaN,NaN,M,1,1.0,V40N,NaN
ASTRAZEN.NS,3552.0,8.16,0.64,0.52,4650.0,1303.0,32.81,31.0,NaN,NaN,M,1,1.0,V40N,NaN
BERGEPAINT.NS,669.0,10.79,-0.17,1.13,858.0,260.0,31.61,28.0,1.0,0.0,M,1,1.0,V40,NaN


In [26]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
RAJESHEXPO.NS,577.0,-13.83,-0.15,-0.69,949.0,440.0,73.08,64.0,NaN,NaN,M,1,1.0,V40N,NaN
SUNTV.NS,448.0,-4.51,0.00,0.69,777.0,249.0,62.31,73.0,NaN,NaN,M,1,1.0,V40N,NaN
ASTRAZEN.NS,3552.0,8.16,0.64,0.52,4650.0,1303.0,32.81,31.0,NaN,NaN,M,1,1.0,V40N,NaN
BERGEPAINT.NS,669.0,10.79,-0.17,1.13,858.0,260.0,31.61,28.0,1.0,0.0,M,1,1.0,V40,NaN
DIXON.NS,4316.0,18.24,0.14,3.88,5750.0,374.0,26.67,33.0,NaN,NaN,M,1,1.0,V40N,NaN


In [27]:
# star + v40/v40n stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
AKZOINDIA.NS,2401.0,7.70,1.12,0.21,2508.0,1360.0,9.32,4.0,NaN,NaN,M,1,1.0,V40,NaN
COLPAL.NS,1648.0,7.14,0.26,0.89,1719.0,931.0,9.01,4.0,1.0,1.0,H,1,1.0,V40,NaN
ABBOTINDIA.NS,22156.0,7.97,0.62,0.20,22892.0,6244.0,4.42,3.0,NaN,NaN,M,1,1.0,V40,NaN
CERA.NS,7834.0,32.38,2.47,2.33,7978.0,2078.0,2.44,2.0,1.0,1.0,H,1,1.0,V40N,NaN
HINDUNILVR.NS,2676.0,4.04,0.12,0.70,2728.0,1406.0,3.93,2.0,1.0,1.0,H,1,1.0,V40,NaN
BAJAJ-AUTO.NS,4756.0,21.79,0.87,2.09,4801.0,1803.0,1.50,1.0,NaN,NaN,L,1,1.0,V40,NaN
EQUITASBNK.NS,87.0,42.70,3.28,2.60,88.0,33.0,1.82,1.0,1.0,1.0,M,1,1.0,V40N,NaN
ICICIBANK.NS,944.0,5.66,0.45,0.87,954.0,252.0,1.42,1.0,1.0,1.0,H,1,1.0,V40,NaN
ITC.NS,445.0,23.38,1.94,1.44,450.0,124.0,1.53,1.0,1.0,1.0,H,1,1.0,V40,NaN


In [28]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANITRANS.NS,823.0,-61.69,-6.28,-2.63,4105.0,147.0,82.92,399.0,1.0,-1.0,L,1,1.0,NaN,NaN
ADANIGREEN.NS,955.0,-37.81,-4.10,-0.11,2970.0,25.0,68.42,211.0,-1.0,-1.0,L,1,1.0,NaN,NaN
ADANIENT.NS,2463.0,-14.05,-0.80,3.48,4165.0,66.0,41.52,69.0,1.0,0.0,M,1,1.0,NaN,NaN
EMAMILTD.NS,380.0,-9.96,-0.95,0.53,588.0,144.0,46.85,55.0,-1.0,1.0,L,1,1.0,V200,NaN
LTIM.NS,4945.0,7.52,0.07,0.60,7439.0,1243.0,40.25,50.0,1.0,0.0,M,1,1.0,V200,NaN
GODREJIND.NS,481.0,8.87,0.00,1.54,644.0,235.0,39.85,34.0,0.0,1.0,L,1,1.0,NaN,NaN
TATAPOWER.NS,222.0,4.62,0.00,1.48,285.0,26.0,24.32,28.0,1.0,0.0,M,1,1.0,NaN,NaN
INDIANB.NS,282.0,9.22,1.94,0.35,346.0,39.0,20.85,23.0,1.0,1.0,H,1,1.0,NaN,NaN
SBICARD.NS,926.0,13.37,-0.12,2.52,1133.0,496.0,32.50,22.0,1.0,0.0,M,1,1.0,NaN,NaN


In [29]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 14


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
BRITANNIA.NS,4940.0,18.01,1.46,1.66,4940.0,1930.0,0.00,0.0,1.0,1.0,H,1,1.0,NaN,NaN
POLYCAB.NS,3584.0,25.09,1.95,2.17,3595.0,526.0,0.36,0.0,1.0,0.0,M,1,1.0,V40N,NaN
TITAN.NS,2912.0,12.67,0.77,1.44,2912.0,733.0,0.00,0.0,1.0,1.0,H,1,1.0,V40,NaN
EQUITASBNK.NS,87.0,42.70,3.28,2.60,88.0,33.0,1.82,1.0,1.0,1.0,M,1,1.0,V40N,NaN
ICICIBANK.NS,944.0,5.66,0.45,0.87,954.0,252.0,1.42,1.0,1.0,1.0,H,1,1.0,V40,NaN
ITC.NS,445.0,23.38,1.94,1.44,450.0,124.0,1.53,1.0,1.0,1.0,H,1,1.0,V40,NaN
LT.NS,2354.0,11.53,1.04,0.71,2384.0,672.0,1.75,1.0,1.0,0.0,M,1,1.0,V200,NaN
BHARTIARTL.NS,834.0,5.48,0.76,1.01,851.0,258.0,2.87,2.0,1.0,1.0,H,1,1.0,NaN,NaN
CERA.NS,7834.0,32.38,2.47,2.33,7978.0,2078.0,2.44,2.0,1.0,1.0,H,1,1.0,V40N,NaN


In [30]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ACC.NS,1847.0,-13.57,-1.03,0.85,2748.0,863.0,47.80,49.0,0.0,0.0,L,0,0.0,NaN,NaN
PVRINOX.NS,1447.0,-10.63,-0.86,-0.75,1917.0,1348.0,82.60,32.0,1.0,-1.0,L,0,0.0,NaN,NaN
PGHH.NS,13978.0,0.72,-0.20,0.05,15778.0,8145.0,23.58,13.0,0.0,0.0,L,0,0.0,V40,NaN
SIEMENS.NS,3734.0,20.62,1.29,1.05,3891.0,834.0,5.14,4.0,1.0,1.0,H,0,0.0,NaN,NaN
NESTLEIND.NS,22493.0,13.83,0.66,1.62,22493.0,8621.0,0.00,0.0,1.0,1.0,H,0,0.0,V40,NaN


In [31]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,


In [32]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,166.0,-4.97,-1.14,1.34,303.0,105.0,69.19,83.0,1.0,1.0,H,1,0.0,NaN,NaN


In [33]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,166.0,-4.97,-1.14,1.34,303.0,105.0,69.19,83.0,1.0,1.0,H,1,0.0,NaN,NaN


In [34]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 20


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
TITAN.NS,2912.0,12.67,0.77,1.44,2912.0,733.0,0.00,0.0,1.0,1.0,H,1,1.0,V40,NaN
BRITANNIA.NS,4940.0,18.01,1.46,1.66,4940.0,1930.0,0.00,0.0,1.0,1.0,H,1,1.0,NaN,NaN
NESTLEIND.NS,22493.0,13.83,0.66,1.62,22493.0,8621.0,0.00,0.0,1.0,1.0,H,0,0.0,V40,NaN
ICICIBANK.NS,944.0,5.66,0.45,0.87,954.0,252.0,1.42,1.0,1.0,1.0,H,1,1.0,V40,NaN
ITC.NS,445.0,23.38,1.94,1.44,450.0,124.0,1.53,1.0,1.0,1.0,H,1,1.0,V40,NaN
EQUITASBNK.NS,87.0,42.70,3.28,2.60,88.0,33.0,1.82,1.0,1.0,1.0,M,1,1.0,V40N,NaN
CERA.NS,7834.0,32.38,2.47,2.33,7978.0,2078.0,2.44,2.0,1.0,1.0,H,1,1.0,V40N,NaN
BHARTIARTL.NS,834.0,5.48,0.76,1.01,851.0,258.0,2.87,2.0,1.0,1.0,H,1,1.0,NaN,NaN
M&M.NS,1373.0,7.78,0.47,1.83,1411.0,262.0,3.31,3.0,1.0,1.0,H,1,1.0,NaN,NaN


In [35]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 13


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
EICHERMOT.NS,3593.0,6.77,0.59,2.14,3850.0,1252.0,9.89,7.0,1.0,1.0,H,1,1.0,V40N,ATH
M&M.NS,1373.0,7.78,0.47,1.83,1411.0,262.0,3.31,3.0,1.0,1.0,H,1,1.0,NaN,NaN
BRITANNIA.NS,4940.0,18.01,1.46,1.66,4940.0,1930.0,0.00,0.0,1.0,1.0,H,1,1.0,NaN,NaN
HDFCLIFE.NS,585.0,7.11,0.18,1.64,757.0,340.0,41.25,29.0,1.0,1.0,H,1,0.0,V40,NaN
NESTLEIND.NS,22493.0,13.83,0.66,1.62,22493.0,8621.0,0.00,0.0,1.0,1.0,H,0,0.0,V40,NaN
TATACONSUM.NS,820.0,8.31,0.13,1.58,866.0,172.0,6.63,6.0,1.0,1.0,H,1,1.0,SS,ATH
MARUTI.NS,9518.0,8.08,0.40,1.49,9736.0,3925.0,3.75,2.0,1.0,1.0,H,1,0.0,NaN,NaN
HAVELLS.NS,1350.0,9.78,0.16,1.44,1456.0,451.0,10.55,8.0,1.0,1.0,H,1,0.0,V40,NaN
TITAN.NS,2912.0,12.67,0.77,1.44,2912.0,733.0,0.00,0.0,1.0,1.0,H,1,1.0,V40,NaN


In [36]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,166.0,-4.97,-1.14,1.34,303.0,105.0,69.19,83.0,1.0,1.0,H,1,0.0,NaN,NaN


In [37]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 45


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
YESBANK.NS,16.0,-3.53,0.00,0.00,383.0,11.0,98.66,2294.0,0.0,0.0,L,1,0.0,NaN,NaN
ADANITRANS.NS,823.0,-61.69,-6.28,-2.63,4105.0,147.0,82.92,399.0,1.0,-1.0,L,1,1.0,NaN,NaN
PEL.NS,807.0,-4.35,-6.52,1.75,2950.0,597.0,91.08,266.0,-1.0,0.0,L,1,0.0,NaN,NaN
ADANIGREEN.NS,955.0,-37.81,-4.10,-0.11,2970.0,25.0,68.42,211.0,-1.0,-1.0,L,1,1.0,NaN,NaN
JCHAC.NS,1042.0,-12.88,-2.01,-0.28,3219.0,1017.0,98.86,209.0,NaN,NaN,M,1,0.0,V40N,NaN
IBREALEST.NS,65.0,-8.66,-1.41,3.23,191.0,39.0,82.89,194.0,-1.0,-1.0,L,1,0.0,NaN,NaN
ZEEL.NS,194.0,-16.04,-0.87,-1.02,542.0,116.0,81.69,179.0,-1.0,1.0,L,1,0.0,NaN,NaN
IEX.NS,125.0,-14.28,-0.68,-0.66,292.0,38.0,65.75,134.0,NaN,NaN,M,0,NaN,NaN,ATH
LALPATHLAB.NS,1988.0,-9.24,-1.00,0.87,4186.0,807.0,65.05,111.0,NaN,NaN,M,1,0.0,V40N,NaN


In [38]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PVRINOX.NS,1447.0,-10.63,-0.86,-0.75,1917.0,1348.0,82.6,32.0,1.0,-1.0,L,0,0.0,NaN,NaN


In [39]:
# save the output
df_prec_dev = df_prec_dev.reset_index()
df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [40]:
# nifty in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-16 00:00:00+05:30,787.0,BHARTIARTL.NS,1.20,0.77,0.39,851.0,258.0,10.79,8.0
2023-05-23 00:00:00+05:30,800.0,BHARTIARTL.NS,2.43,0.77,0.65,851.0,258.0,8.60,6.0
2023-05-30 00:00:00+05:30,818.0,BHARTIARTL.NS,4.26,0.89,0.51,851.0,258.0,5.56,4.0
2023-06-06 00:00:00+05:30,829.0,BHARTIARTL.NS,5.26,0.89,1.02,851.0,258.0,3.71,3.0
2023-06-13 00:00:00+05:30,834.0,BHARTIARTL.NS,5.48,0.76,1.01,851.0,258.0,2.87,2.0
